In [15]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr

In [25]:
load_dotenv(override=True)

openai_api = os.getenv("OPENAI_API_KEY")
serper_api = os.getenv("X-API-KEY")

if openai_api:
    print(f"Exists and starts with {openai_api[:10]}")

if serper_api:
    print(f"Exists and starts with {serper_api[:10]}")

Exists and starts with sk-proj-61
Exists and starts with db85bd4af6


In [19]:
openai = OpenAI()
MODEL = "gpt-5-mini"

In [17]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [8]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(city):
    print(f"Tool called for {city.lower()}")
    price = ticket_prices.get(city.lower(), "Unknown city")
    return f"The price for {city.lower()} is {price}"

In [10]:
get_ticket_price('PARIS')

Tool called for paris


'The price for paris is $899'

In [13]:
price_function = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Returns the cost of a round-trip flight to a specific city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city name, e.g., 'London'"
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

In [22]:
tools = [price_function]

In [23]:
def handle_tool_calls(message):
    response = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get("destination_city")
            price_details = get_ticket_price(city)
            response.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })

    return response

In [20]:
def chat(message, history):
    history = [{"role":h[ "role"], "content":h ["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [24]:
gr.ChatInterface(fn=chat, type="messages", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called for paris
Tool called for paris
Tool called for london
Tool called for tokyo
Tool called for albania
